In [40]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from scipy import sparse

In [41]:
book_df = pd.read_csv('data/Books.csv')
ratings_df = pd.read_csv('data/Ratings.csv')
user_df = pd.read_csv('data/Users.csv')
ratings_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
ratings_df = ratings_df[ratings_df['Book-Rating'] > 0]
ratings_df = ratings_df[ratings_df['User-ID'].isin(ratings_df['User-ID'].unique()[0:10000])]
ratings_df.shape

(61812, 5)

In [43]:
book_user_rating = book_df.merge(ratings_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)
book_user_rating.head()

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,8,5
1,0002005018,Clara Callan,Richard Bruce Wright,11676,8
2,0060973129,Decision in Normandy,Carlo D'Este,2954,8
3,0399135782,The Kitchen God's Wife,Amy Tan,11676,9
4,0399135782,The Kitchen God's Wife,Amy Tan,29526,9


In [44]:
d = {}
for i, j in enumerate(book_user_rating.ISBN.unique()):
    d[j] = i
book_user_rating["unique_id_book"] = book_user_rating["ISBN"].map(d)
book_user_rating.reset_index(drop=True, inplace=True)
book_user_rating.head()


,ISBN,Book-Title,Book-Author,User-ID,Book-Rating,unique_id_book
0,0002005018,Clara Callan,Richard Bruce Wright,8,5,0
1,0002005018,Clara Callan,Richard Bruce Wright,11676,8,0
2,0060973129,Decision in Normandy,Carlo D'Este,2954,8,1
3,0399135782,The Kitchen God's Wife,Amy Tan,11676,9,2
4,0399135782,The Kitchen God's Wife,Amy Tan,29526,9,2


test
s1
Строим разреженную матрицу

In [45]:
df_pivot = book_user_rating.pivot(index='unique_id_book',
                                  columns='User-ID',
                                  values='Book-Rating').fillna(0)

In [46]:
df_pivot.head()

User-ID,8,9,12,14,16,17,19,22,26,32,...,278831,278832,278836,278843,278844,278846,278849,278851,278852,278854
unique_id_book,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df_pivot.shape


(35582, 8709)

In [48]:
df_sparse = sparse.csr_matrix(df_pivot)
df_pivot.reset_index(inplace=True)

In [49]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(df_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [50]:
ISBN = '0399135782'
book_unique_id = book_user_rating[book_user_rating['ISBN']==ISBN]['unique_id_book'].unique()[0]
book_unique_id

2

In [51]:
def get_book_recommendation(ISBN, n_recommend: int):
    if ISBN in book_user_rating['ISBN']:
        book_unique_id = book_user_rating[book_user_rating['ISBN']==ISBN]['unique_id_book'].unique()[0]
        distances, indices = knn.kneighbors(df_sparse[book_unique_id], n_neighbors=n_recommend)
        rec_book_indices = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())),
                                  key=lambda x: x[1])[:0:-1]
        recommendations = []
        for val in rec_book_indices:
            book_idx = book_user_rating.iloc[val[0]]['ISBN']
            idx = book_df[book_df['ISBN'] == book_idx].index
            recommendations.append({'ISBN':book_df.iloc[idx]['ISBN'].values[0],
                                    'Book-Title':book_df.iloc[idx]['Book-Title'].values[0],
                                    'Book-Author':book_df.iloc[idx]['Book-Author'].values[0],
                                    'Year-Of-Publication':book_df.iloc[idx]['Year-Of-Publication'].values[0],
                                    'Publisher':book_df.iloc[idx]['Publisher'].values[0],
                                    'Distance':val[1]})
        df = pd.DataFrame(recommendations, index=range(1, n_recommend))
        return df
    else:
        return "Book not found."

def recommend(user_id, n_recommend=5):
    ISBN = book_user_rating.loc[book_user_rating[book_user_rating['User-ID'] == user_id]['Book-Rating'].idxmax()]['ISBN']
    recommendations = get_book_recommendation(ISBN, n_recommend)

    return recommendations

[3, 2]